In [2]:
import pandas as pd

df = pd.read_csv('MELD.Raw/dev_sent_emo.csv',encoding='utf-8')
df['Utterance'] = df.Utterance.str.replace('',"'")
# df['gender'] = df.Speaker.apply(lambda x: 'male' if x == 'Ross' or x == 'Joey' or x == 'Chandler' else 'female')
df = df.sample(frac=0.2,ignore_index=True)
df

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,461,You tell them to wait!,Ross,anger,negative,42,1,5,1,"00:21:11,937","00:21:14,230"
1,778,"Oh, fine! Take his side!",Phoebe,anger,negative,72,18,5,11,"00:12:13,774","00:12:16,234"
2,451,"Okay Joey, first of all Kash Ford is not peopl...",Rachel,anger,negative,40,6,8,5,"00:06:56,874","00:06:58,750"
3,792,Seriously. What?!,Rachel,surprise,negative,74,10,8,3,"00:10:18,909","00:10:20,827"
4,861,No. Have you seen David?,Max,neutral,neutral,80,1,1,10,"00:12:04,807","00:12:06,683"
...,...,...,...,...,...,...,...,...,...,...,...
217,771,"Yeah, well, y'know maybe you just need to try ...",Phoebe,anger,negative,72,11,5,11,"00:11:43,911","00:11:47,914"
218,105,It's a witness not a perp. And no one talks li...,Gary,anger,negative,10,4,5,20,"0:07:07,593","0:07:10,096"
219,815,"So it's kinda like, you're, you know.",Chandler,neutral,neutral,76,12,3,1,"00:17:00,769","00:17:08,860"
220,445,I thought I was a complete idiot.,Rachel,surprise,positive,40,0,8,5,"00:06:37,605","00:06:40,857"


In [3]:
from nltk.stem import WordNetLemmatizer
import re


def preprocess(text):
    t = text.lower()
    t = re.sub('\d+',r'',t)
    t = re.sub(r'\W+',r' ',t)
    return t

lemmatizer = WordNetLemmatizer()



df['prepro'] = [' '.join([lemmatizer.lemmatize(preprocess(txt))])
                 .strip() for txt in df['Utterance']]

In [4]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(categories=[df.Emotion.unique()])
labels = ohe.fit_transform(df.Emotion.to_numpy().reshape(-1,1)).toarray()

In [78]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, TFBertModel
import numpy as np

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name='input_ids')
token_type_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name='token_type_ids')
attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name='attention_mask')

bert_outputs = model(
    input_ids=input_ids,
    token_type_ids=token_type_ids,
    attention_mask=attention_mask
)

pooled_output = bert_outputs.pooler_output
dense_layer = tf.keras.layers.Dense(256, activation='relu')(pooled_output)
output_layer = tf.keras.layers.Dense(7, activation='softmax')(dense_layer)


custom_model = tf.keras.Model(
    inputs=[input_ids, token_type_ids, attention_mask],
    outputs=output_layer
)


# Prepare dataset for text classification
train_texts = list(df.prepro.values)
train_labels = list(labels)

# Tokenize input texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

# Convert labels to tensors
train_labels = tf.convert_to_tensor(train_labels)

train_inputs = {
    'input_ids': np.array(train_encodings['input_ids']),
    'token_type_ids': np.array(train_encodings['token_type_ids']),
    'attention_mask': np.array(train_encodings['attention_mask'])
}

# Create TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    train_inputs,
    train_labels
)).shuffle(len(train_inputs)).batch(16,drop_remainder=True)  # Adjust batch size as needed

# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

# Compile the model
custom_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Fine-tune the model
custom_model.fit(train_dataset, epochs=3, verbose=1)  # Adjust epochs as needed


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/3
13/13 [==============================] - 47s 2s/step - loss: 1.8082 - accuracy: 0.3317
Epoch 2/3
13/13 [==============================] - 29s 2s/step - loss: 1.4398 - accuracy: 0.4423
Epoch 3/3
13/13 [==============================] - 29s 2s/step - loss: 1.2264 - accuracy: 0.5577


In [82]:
# ... (Your existing code remains the same)

# Fine-tune the model
custom_model.fit(train_dataset, epochs=3, verbose=1)  # Adjust epochs as needed

# Extract embeddings from the model
embeddings_model = tf.keras.Model(inputs=custom_model.input, outputs=bert_outputs.last_hidden_state)

# Prepare data for inference
test_texts = ["Your test sentences here"]
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Convert inputs to tensors
test_inputs = {
    'input_ids': np.array(test_encodings['input_ids']),
    'token_type_ids': np.array(test_encodings['token_type_ids']),
    'attention_mask': np.array(test_encodings['attention_mask'])
}

# Get embeddings for test data
embeddings = embeddings_model.predict(test_inputs)

# 'embeddings' now contains the embeddings for the test data
print(embeddings)


Epoch 1/3


13/13 [==============================] - 31s 2s/step - loss: 0.9785 - accuracy: 0.6827
Epoch 2/3
13/13 [==============================] - 31s 2s/step - loss: 0.7846 - accuracy: 0.7692
Epoch 3/3
1/1 [==============================] - 3s 3s/step
(1, 6, 768)


In [68]:
blue = tokenizer('i am sad',return_tensors='tf')
blue

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 1045, 2572, 6517,  102]])>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]])>}

In [74]:
pred = custom_model.predict([blue['input_ids'],blue['token_type_ids'],blue['attention_mask']])
pred

1/1 [==============================] - 4s 4s/step


array([[0.13594277, 0.08063605, 0.47514027, 0.18977657, 0.01977132,
        0.03838995, 0.06034305]], dtype=float32)

In [27]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

# Load fine-tuned BERT model
model_name = 'bert-base-uncased'  # or your fine-tuned model name
tokenizer = BertTokenizer.from_pretrained(model_name)
amodel = TFBertModel.from_pretrained(model_name)

# Your dataset sentences
sentences = df.prepro.values

# Tokenize the sentences
tokenized_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='tf')

# Get embeddings
outputs = amodel(tokenized_inputs)
hidden_states = outputs[0]  # Extract hidden states from the output tuple

# Print keys in the outputs object
print(outputs.keys())

# Check the shape of hidden states
print(hidden_states.shape)  # Print shape of hidden states


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


odict_keys(['last_hidden_state', 'pooler_output'])
(5, 4, 768)


In [31]:
model(blue)

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(1, 5, 768), dtype=float32, numpy=
array([[[-0.19910356,  0.538442  , -0.220623  , ...,  0.01834938,
          0.32222158,  0.2692385 ],
        [-0.57449394,  0.39249668, -0.5139347 , ...,  0.120591  ,
          0.809383  ,  0.12798725],
        [-0.77094674,  0.33647695, -0.1416466 , ...,  0.23776443,
          0.29806218,  0.23093735],
        [-1.0086286 ,  0.46211925, -0.00540426, ...,  0.67257756,
          0.18979183, -0.22161974],
        [ 0.6236303 ,  0.16630654, -0.37921596, ..., -0.03854161,
         -0.4813545 , -0.4720742 ]]], dtype=float32)>, pooler_output=<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-8.91511977e-01, -3.56628299e-01, -2.96447903e-01,
         7.63155341e-01,  6.84296596e-04, -5.61037362e-02,
         8.69579017e-01,  2.51043111e-01, -3.83736402e-01,
        -9.99966323e-01,  4.73340973e-03,  8.46770823e-01,
         9.84883368e-01,  1.14933394e-01,  9.44804251